In [9]:
def lexer(input_str):
    #? AUTOMATA
    transiciones = {
        'q0': [
            (lambda c: c == '$', 'q1'),
            (lambda c: c.isdigit(), 'q3'),
            (lambda c: c == "'", 'q6'),
            (lambda c: c == '/', 'q8'),
            (lambda c: c == ':', 'q12'),
            (lambda c: c == '<', 'q14'),
            (lambda c: c == '>', 'q16'),
            (lambda c: c == '!', 'q18'),
            (lambda c: c == '=', 'q20'),
            (lambda c: c == '+', 'q22'),
            (lambda c: c == '-', 'q23'),
            (lambda c: c == '*', 'q24'),
            (lambda c: c == '|', 'q25'),
            (lambda c: c == '&', 'q27'),
            (lambda c: c == '{', 'q29'),
            (lambda c: c == '}', 'q30'),
            (lambda c: c == '[', 'q31'),
            (lambda c: c == ']', 'q32'),
            (lambda c: c == '(', 'q33'),
            (lambda c: c == ')', 'q34'),
            (lambda c: c == ';', 'q35'),
        ],
        'q1': [(lambda c: c.isalnum(), 'q2')],
        'q2': [(lambda c: c.isalnum(), 'q2')],
        'q3': [
            (lambda c: c.isdigit(), 'q3'),
            (lambda c: c == '.', 'q4'),
        ],
        'q4': [(lambda c: c.isdigit(), 'q5')],
        'q5': [(lambda c: c.isdigit(), 'q5')],
        'q6': [
            (lambda c: c != "'", 'q6'),
            (lambda c: c == "'", 'q7'),
        ],
        'q7': [],
        'q8': [(lambda c: c == '*', 'q9')],
        'q9': [
            (lambda c: c != '*', 'q9'),
            (lambda c: c == '*', 'q10'),
        ],
        'q10': [(lambda c: c == '/', 'q11')],
        'q12': [(lambda c: c == '=', 'q14')],
        'q14': [(lambda c: c == '=', 'q15')],
        'q16': [(lambda c: c == '=', 'q17')],
        'q18': [(lambda c: c == '=', 'q19')],
        'q20': [(lambda c: c == '=', 'q21')],
        'q25': [(lambda c: c == '|', 'q26')],
        'q27': [(lambda c: c == '&', 'q28')],
    }

    #? Arreglo para consultar si es o no un estado final
    estados_aceptacion = {
        'q2': 100,  # identificador
        'q3': 150,  # entero
        'q5': 200,  # decimal
        'q7': 250,  # string
        'q8': 450,  # aritmetico (/)
        'q11': 300,  # comentario
        'q12': 550,  # delimitador (:)
        'q13': 350,  # asignacion
        'q14': 400,  # relacional
        'q15': 400,  # relacional
        'q16': 400,  # relacional
        'q17': 400,  # relacional
        'q18': 500,  # logico
        'q19': 400,  # relacional
        'q21': 400,  # relacional
        'q22': 450,  # aritmetico
        'q23': 450,  # aritmetico
        'q24': 450,  # aritmetico
        'q26': 500,  # logico
        'q28': 500,  # logico
        'q29': 550,  # delimitador
        'q30': 550,  # delimitador
        'q31': 550,  # delimitador
        'q32': 550,  # delimitador
        'q33': 550,  # delimitador
        'q34': 550,  # delimitador
        'q35': 550,  # delimitador
    }

    #? Arreglo para consultar el significado del token
    codigos = {
        100: 'identificador',
        150: 'entero',
        200: 'decimal',
        250: 'string',
        300: 'comentario',
        350: 'asignacion',
        400: 'relacional',
        450: 'aritmetico',
        500: 'logico',
        550: 'delimitador',
    }

    index = 0
    resultados = []
    longitud = len(input_str)

    while index < longitud:
        #! Ignorar  los T400, que son los espacios, tabuladores y saltos de linea
        if input_str[index] in {' ', '\t', '\n'}:
            index += 1 #! salto de cursor
            continue
        #! Cada iteracion, el cursor va a volver a 0
        estado_actual = 'q0' #? Valor que el cursor esta leyendo
        inicio = index
        estado_aceptacion = None #? Se guarda el estado de aceptacion
        pos_aceptado = -1 #? Se guarda el index al que pertenece el estado de aceptacion, "posicion aceptado"

        # Procesar hasta encontrar un delimitador o que ya no hay mas transiciones
        while index < longitud:
            char = input_str[index]
            transicion_encontrada = False

            # Verificar transiciones para el estado actual
            for condicion, estado_destino in transiciones.get(estado_actual, []):
                if condicion(char):
                    estado_actual = estado_destino
                    transicion_encontrada = True
                    break
            # Caso en el que no hay transiciones
            if not transicion_encontrada:
                break  

            # Verificar si el nuevo estado es de aceptacion
            if estado_actual in estados_aceptacion:
                estado_aceptacion = estado_actual
                pos_aceptado = index

            index += 1

        #? Manejar resultados despues de procesar un solo token
        if estado_aceptacion is not None:
            # Token reconocido
            token = input_str[inicio:pos_aceptado + 1]
            codigo = estados_aceptacion[estado_aceptacion]
            resultados.append(f"{token} = {codigos[codigo]}")
            index = pos_aceptado + 1
        else:
            # Manejar errores
            if index == inicio:
                # No hubo ninguna transición desde q0
                resultados.append(f"{input_str[inicio]} = token no existe")
                index += 1
            else:
                # Token malformado
                token = input_str[inicio:index]
                resultados.append(f"{token} = token no reconocido")

    return resultados

# entrada = "$name . 200.0 'hola' /*comentario*/ := <= >= != == || && { } + - * /"
# * Leyendo el txt
with open('codigo.txt', 'r') as file:
    entrada = file.read()
resultados = lexer(entrada)

#// Output en terminal
# for res in resultados:
#     print(res)

# * Escribiendo el output
with open('./output.txt', 'w') as file:
    file.write('\n'.join(resultados))